# Process COHD Clinical Data

Jupyter Notebook to download and preprocess files for the [COHD Clinical Data](http://cohd.smart-api.info/) transformation to BioLink RDF.

Sample files are [available on GitHub](https://github.com/MaastrichtU-IDS/d2s-scripts-repository/tree/master/resources/cohd-sample). The complete data comes with a 27G TSV file.

### Download files

In [2]:
import os
import glob
import requests
import functools
import shutil
import pandas as pd 

def download_file(url):
    local_filename = url.split('/')[-1]
    with requests.get(url, stream=True) as r:
        with open(local_filename, 'wb') as f:
            r.raw.read = functools.partial(r.raw.read, decode_content=True)
            shutil.copyfileobj(r.raw, f)
    print(local_filename + ' downloaded.')
    if local_filename.endswith('.gz') or local_filename.endswith('.zip'):
        shutil.unpack_archive(local_filename, '.')
    return local_filename

def convert_tsv_to_csv(tsv_file):
    csv_table=pd.read_table(tsv_file,sep='\t')
    csv_table.to_csv(tsv_file[:-4] + '.csv',index=False)

# Variables and path for the dataset
dataset_id = 'cohd'
dsri_flink_pod_id = 'flink-jobmanager-5f9db544c8-qswmg'
input_folder = '/notebooks/workspace/input/' + dataset_id
mapping_folder = '/notebooks/datasets/' + dataset_id + '/mapping'
os.makedirs(input_folder, exist_ok=True)

In [ ]:
# Use input folder as working folder
os.chdir(input_folder)

# Complete 27G dataset:
files_to_download = [
    'https://filedn.com/ll1efYfBhLaV67ONaCyMlKh/cohd-v2.tar.gz'
]

# Download each file and uncompress them if needed
# Use Bash because python fails
for download_url in files_to_download:
    os.system('wget -N ' + download_url)
    os.system('find . -name "*.tar.gz" -exec tar -xzvf {} \;')
#   download_file(download_url)

# Rename .txt to .tsv
listing = glob.glob('*.txt')
for filename in listing:
    os.rename(filename, filename[:-4] + '.tsv')

    
# Convert the 27G TSV to CSV to be processed with the RMLStreamer
convert_tsv_to_csv('paired_concept_counts_associations.tsv')
cmd_convert_csv = """sed -e 's/"/\\"/g' -e 's/\t/","/g' -e 's/^/"/' -e 's/$/"/'  -e 's/\r//' paired_concept_counts_associations.tsv > paired_concept_counts_associations.csv"""
os.system(cmd_convert_csv)

### Split the large associations CSV file

In [13]:
with open('paired_concept_counts_associations.csv') as f:
    csv_header = f.readline().strip() 

shutil.rmtree('split', ignore_errors=True)
os.makedirs('split', exist_ok=True)

# Using bash command to split, as it is more efficient for those operations
# split_line_count = '100'
# Split 27G file in 60 chunks
split_line_count = '4106657'
os.system('split -l ' + str(split_line_count) + ' paired_concept_counts_associations.csv split/paired_concept_counts_associations-split_')

# Remove the header line (bash command)
os.system('sed -i -e "1d" split/paired_concept_counts_associations-split_aa')

# Iterate over splitted files
shutil.rmtree(mapping_folder + '/split', ignore_errors=True)
os.makedirs(mapping_folder + '/split', exist_ok=True)
count = 0
listing = glob.glob('split/paired_*')
for filename in listing:
    # Rename files from split to set .csv file extension
    os.rename(filename, 'split/paired_concept_counts_associations_' + str(count) + '.csv')
    split_rml_file = 'split/associations-mapping-' + str(count) + '.rml.ttl'
    # Copy RML mapping file in mapping/split folder, and replace the file names with count index
    shutil.copyfile(mapping_folder + '/associations-mapping.rml.ttl', mapping_folder + '/' + split_rml_file)
    with open(mapping_folder + '/' + split_rml_file) as f:
        file_content = f.read()
    file_content = file_content.replace('paired_concept_counts_associations.csv', 'paired_concept_counts_associations_' + str(count) + '.csv')
    with open(mapping_folder + '/' + split_rml_file, "w") as f:
        f.write(file_content)
    # Print command to run on OpenShift DSRI
    #print('oc exec ' + dsri_flink_pod_id + ' -- /opt/flink/bin/flink run -c io.rml.framework.Main /mnt/RMLStreamer.jar --path /mnt/datasets/cohd/mapping/split/associations-mapping-' + str(count) + '.rml.ttl --outputPath /mnt/workspace/import/openshift-rmlstreamer-associations-mapping_rml_ttl-cohd-' + str(count) + '.nt --job-name "[d2s] RMLStreamer associations-mapping.rml.ttl - cohd ' + str(count) + '"')
    # To run detached in the flink manager pod:
    print('nohup /opt/flink/bin/flink run -c io.rml.framework.Main /mnt/RMLStreamer.jar --path /mnt/datasets/cohd/mapping/split/associations-mapping-' + str(count) + '.rml.ttl --outputPath /mnt/workspace/import/openshift-rmlstreamer-associations-mapping_rml_ttl-cohd-' + str(count) + '.nt --job-name "[d2s] RMLStreamer associations-mapping.rml.ttl - cohd ' + str(count) + '" &')
    count += 1

    
# Add header line to each split file (bash command)
add_header_cmd = """sed -i '1s/^/""" + csv_header + """\\n/' split/*.csv"""
os.system(add_header_cmd)

nohup /opt/flink/bin/flink run -c io.rml.framework.Main /mnt/RMLStreamer.jar --path /mnt/datasets/cohd/mapping/split/associations-mapping-0.rml.ttl --outputPath /mnt/workspace/import/openshift-rmlstreamer-associations-mapping_rml_ttl-cohd-0.nt --job-name "[d2s] RMLStreamer associations-mapping.rml.ttl - cohd 0" &
nohup /opt/flink/bin/flink run -c io.rml.framework.Main /mnt/RMLStreamer.jar --path /mnt/datasets/cohd/mapping/split/associations-mapping-1.rml.ttl --outputPath /mnt/workspace/import/openshift-rmlstreamer-associations-mapping_rml_ttl-cohd-1.nt --job-name "[d2s] RMLStreamer associations-mapping.rml.ttl - cohd 1" &
nohup /opt/flink/bin/flink run -c io.rml.framework.Main /mnt/RMLStreamer.jar --path /mnt/datasets/cohd/mapping/split/associations-mapping-2.rml.ttl --outputPath /mnt/workspace/import/openshift-rmlstreamer-associations-mapping_rml_ttl-cohd-2.nt --job-name "[d2s] RMLStreamer associations-mapping.rml.ttl - cohd 2" &
nohup /opt/flink/bin/flink run -c io.rml.framework.Main

In [ ]:
# Copy file to Pod (needs to be executed on a machine with oc installed)
# oc cp workspace/input/cohd/split.zip flink-jobmanager-5f9db544c8-qswmg:/mnt/workspace/input/cohd/paired_concept_counts_associations-split.zip

# Copy back to node2 from DSRI Flink (using ZSH)
# nohup oc cp flink-jobmanager-5f9db544c8-qswmg:/mnt/workspace/import workspace/graphdb-import/cohd &!

## Process and load concepts

For `concepts.tsv`, `datasets.tsv` and `single_concepts.counts.tsv` We will use CWL workflows to integrate data with SPARQL queries. The structured data is first converted to a generic RDF based on the data structure, then mapped to BioLink using SPARQL. The SPARQL queries are defined in `.rq` files and can be [accessed on GitHub](https://github.com/MaastrichtU-IDS/d2s-project-template/tree/master/datasets/cohd/mapping).

Start the required services (here on our server, defined by the `-d trek` arg):

```bash
d2s start tmp-virtuoso drill -d trek
```

Run the following d2s command in the d2s-project folder:

```bash
d2s run csv-virtuoso.cwl cohd
d2s run csv-virtuoso.cwl drugbank
d2s run csv-virtuoso.cwl omop-drugbank
```

[HCLS metadata](https://www.w3.org/TR/hcls-dataset/) can be computed for the COHD graph after the associations data has ben uploaded (find associations as BioLink RDF on our server in `/data/graphdb-import/trek/cohd`)

```bash
d2s run compute-hcls-metadata.cwl cohd
d2s run compute-hcls-metadata.cwl drugbank
d2s run compute-hcls-metadata.cwl omop-drugbank
```


## Load the BioLink model

Load the [BioLink model ontology as Turtle](https://github.com/biolink/biolink-model/blob/master/biolink-model.ttl) in the graph `https://w3id.org/biolink/biolink-model` in the triplestore


## Reconvert COHD

Update RMLStreamer.jar:
```
wget -O RMLStreamer.jar https://github.com/RMLio/RMLStreamer/releases/download/v2.0.0/RMLStreamer-2.0.0.jar
```

Re-run with parallelism:
```
nohup /opt/flink/bin/flink run -p 128 -c io.rml.framework.Main /mnt/RMLStreamer.jar toFile -m /mnt/datasets/cohd/mapping/cohd-associations.rml.ttl -o /mnt/workspace/import/openshift-rmlstreamer-cohd-associations.nq --job-name "[d2s] RMLStreamer cohd-associations.rml.ttl" &
```

Check if running:
```
oc rsh flink-jobmanager-7459cc58f7-cjcqf
ls -alh /mnt/workspace/import/openshift-rmlstreamer-cohd-associations.nq
```

## Merge and send output

```
cd /mnt/workspace/import/openshift-rmlstreamer-cohd-associations.nq
nohup cat * >> openshift-rmlstreamer-cohd-associations.nq &

ls -alh /mnt/workspace/import/openshift-rmlstreamer-cohd-associations.nq/openshift-rmlstreamer-cohd-associations.nq
```

Zip the merged output file:

```
nohup gzip openshift-rmlstreamer-cohd-associations.nq &
```

## Copy to node2

SSH connect to node2, http_proxy var need to be changed temporary to access DSRI

```
export http_proxy=""
export https_proxy=""
```

Copy with `oc` tool:
```
oc login
oc cp flink-jobmanager-7459cc58f7-cjcqf:/mnt/workspace/import/openshift-rmlstreamer-cohd-associations.nq/openshift-rmlstreamer-cohd-associations.nq.gz /data/graphdb/import/umids-download &!
```

Check (19G total):
```
ls -alh /data/graphdb/import/umids-download
cp /data/graphdb/import/umids-download/openshift-rmlstreamer-cohd-associations.nq.gz /data/d2s-project-trek/workspace/dumps/rdf/cohd/
gzip -d openshift-rmlstreamer-cohd-associations.nq.gz
```

Reactivate the proxy (`EXPORT http_proxy`)

## Preload to GraphDB

Check the generated COHD file on node2 at:

```
cd /data/d2s-project-trek/workspace/dumps/rdf/cohd
```

Replace wrong triples:

```
sed -i 's/"-inf"^^<http:\/\/www.w3.org\/2001\/XMLSchema#double>/"-inf"/g' openshift-rmlstreamer-cohd-associations.nq
```

Start preload:

```
cd /data/deploy-ids-services/graphdb/preload-cohd
docker-compose up -d
```

The COHD repository will be created in `/data/graphdb-preload/data`, copy it to the main GraphDB:

```
mv /data/graphdb-preload/data/repositories/cohd /data/graphdb/data/repositories
```

## Split in 10 to solve load issue

```
wc -l openshift-rmlstreamer-cohd-associations.nq > cohd_line_count.txt &!
ps aux | grep "wc -l"
```

Divide line count by 10

```
split -l 93084203 *.nq split/
```

Split in 10:
```
split -l 298033553 --additional-suffix .nq openshift-rmlstreamer-cohd-associations.nq split/openshift-rmlstreamer-cohd-associations- &!
```

Check it:

```
ps aux | grep "split -l"
```